<a href="https://colab.research.google.com/github/MatthewNg3416/-/blob/master/MWN_240715.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# 240714


from email.mime import image
import torchvision.transforms as transforms
from torchvision.datasets import VOCSegmentation
from torch.utils.data import DataLoader

import torch.optim as optim

from torchvision.utils import save_image

import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )

        self.encoder1 = conv_block(in_channels, 64)
        self.encoder2 = conv_block(64, 128)
        self.encoder3 = conv_block(128, 256)
        self.encoder4 = conv_block(256, 512)
        self.bottleneck = conv_block(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = conv_block(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = conv_block(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = conv_block(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = conv_block(128, 64)

        self.final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(F.max_pool2d(enc1, kernel_size=2, stride=2))
        enc3 = self.encoder3(F.max_pool2d(enc2, kernel_size=2, stride=2))
        enc4 = self.encoder4(F.max_pool2d(enc3, kernel_size=2, stride=2))
        bottleneck = self.bottleneck(F.max_pool2d(enc4, kernel_size=2, stride=2))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        return self.final(dec1)



if __name__ == "__main__":
    print("unet:")
    model=UNet(in_channels=3,
               out_channels=1)


    print(next(model.parameters()).device)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print(next(model.parameters()).device)

    # 数据预处理
    transform=transforms.Compose([
        transforms.Resize((128,128)),
        transforms.ToTensor()
    ])

    # 加载Pascal VOC数据集
    train_dataset=VOCSegmentation(
        root='./data',
        year='2012',
        image_set='train',
        download=True,
        transform=transform,
        target_transform=transform
    )
    train_loader=DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True)

    criterion=nn.BCEWithLogitsLoss()
    optimizer=optim.Adam(model.parameters(),
                         lr=.001)
    # 训练循环
    num_epochs=10
    for epoch in range(num_epochs):
        model.train()
        epoch_loss=0
        for images,masks in train_loader:
            images=images.to(device)
            masks=masks.to(device)
            optimizer.zero_grad()
            outputs=model(images)
            loss=criterion(outputs,masks)
            loss.backward()
            optimizer.step()
            epoch_loss+=loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, \
              Loss: {epoch_loss/len(train_loader)}")



    # print(model)

    # 模型评估
    model.eval()
    i=0
    with torch.no_grad():
        for images,masks in train_loader:
            images=images.to(device)
            masks=masks.to(device)
            outputs=model(images)
            #将结果保存为图片
            save_image(outputs,
                     'output{i}.png')
            i+=1
            if(i>10):
                break






unet:
cpu
cpu


100%|██████████| 1999639040/1999639040 [01:11<00:00, 27907057.33it/s]


Extracting ./data/VOCtrainval_11-May-2012.tar to ./data


KeyboardInterrupt: 

In [23]:
print("unet----")
print("----")
import torch.nn as nn
class Unet2(nn.Module):
  def __init__(self,in_channels,out_channels) -> None:
      super(Unet2,self).__init__()

      self.prt="prt"
      print("Unet init")

unet2 = Unet2(2,3)
print(unet2.parameters)
a = torch.tensor(2)
b = torch.tensor(3)
c = a+b
import pdb
# pdb.set_trace()
print(c)






unet----
----
Unet init
<bound method Module.parameters of Unet2()>
tensor(5)


In [ ]:

#
import torch
from matplotlib import pyplot as plt

def plot_curve(data):
  fig=plt.figure()
  plt.plot(range(len(data)),
           data,
           color='blue')
  plt.legend(['value'],
      loc="upper right")
  plt.xlabel('step')
  plt.ylabel('value')
  plt.show()


def plot_image(img,
    label,
    name):
  fig=plt.figure()
  for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(img[i]]0]*0.3081+0.1307,cmap='gray',interpolation='none')
    plt.title("{}:{}".format(name,label))
    plt.xticks([])
    plt.yticks([])
  plt.show()


  def one_hot(label,depth=10):
    out=torch.zeros(label.size(0),depth)
    idx=torch.LongTensor(label).view(-1,1)
    out.scatter_(dim=1,
          index=idx,
          value=1)
    return out








